# Samhliðamálheild fyrir ensku og íslensku - ParIce
Hér er unnið með ParIce málheildina. Ýmsar villur í samhliða málheildinni eru lagaðar og hún er síuð.

Gert er ráð fyrir að eftirfarandi gögn séu til staðar
- ParIce staðlað, `source_dir/(train|dev|test).(en|is)`.

Eftir að hafa keyrt reikniritið verður til `target_dir/(train|dev|test).(en|is)` þar sem öll skrefin hafa verið útfærð.

In [39]:
# The location of Parice to read
source_dir = '/work/haukurpj/data/formatted/Parice1.0'

# The location of filtered ParIce dataset, provided by Miðeind.
source_test_dir = '/work/haukurpj/data/raw/parice_test_set_filtered'

# The location of where to write the results
target_dir = '/work/haukurpj/data/filtered/Parice1.0'

from glob import glob
from pprint import pprint

In [40]:
import pathlib

source_dir = pathlib.Path(source_dir)
source_test_dir = pathlib.Path(source_test_dir)
target_dir = pathlib.Path(target_dir)
assert source_dir.exists()
assert source_test_dir.exists()
if not target_dir.exists():
    target_dir.mkdir()

In [41]:
def read_file(path):
    corpus = []
    with path.open() as f_in:
        return [line for line in f_in]
train_is = read_file(source_dir.joinpath('train.is'))
train_en = read_file(source_dir.joinpath('train.en'))
dev_is = read_file(source_dir.joinpath('dev.is'))
dev_en = read_file(source_dir.joinpath('dev.en'))
test_ees_is = read_file(source_dir.joinpath('test-ees.is'))
test_ees_en = read_file(source_dir.joinpath('test-ees.en'))
test_ema_is = read_file(source_dir.joinpath('test-ema.is'))
test_ema_en = read_file(source_dir.joinpath('test-ema.en'))
test_opensubtitles_is = read_file(source_dir.joinpath('test-opensubtitles.is'))
test_opensubtitles_en = read_file(source_dir.joinpath('test-opensubtitles.en'))

In [42]:
print(len(train_is), len(train_en))

3256945 3256945


## Laga setningabrot
Á nokkrum stöðum í ParIce textanum vantar bil á eftir punkti og tilreiðirinn nær ekki að brjóta þær setningar. Samsvarandi setningar í ensku eiga líka við þetta vandamál, en í minna mælli. Við reynum að vera íhaldsöm og setjum bara bil á milli þegar hástafur kemur eftir punkt á undan tveimur stöfum.

Dæmi:

In [43]:
import re

def print_indices(corpus, indices):
    indices_list = list(indices)
    indices_list.sort()
    
    len_indices = len(indices_list)
    found = 0
    for index, line in enumerate(corpus):
        if found == len_indices:
            return
        if index == indices_list[found]:
            print(line)
            found += 1
    

def find_indices_match(corpus, pattern, max_count=10):
    count = 0
    found = []
    for index, line in enumerate(corpus):
        match = re.match(pattern, line)
        if match:
            if max_count > count:
                count += 1
                found.append(index)
            else:
                break
    return found

sent_break_indices = find_indices_match(train_is, re.compile(r'([\w\(\)\[\]\.]{2,})\.([A-ZÁÐÉÍÓÚÝÞÆÖ])'), max_count=10)
print_indices(train_is, sent_break_indices)

3b.GERÐARVIÐURKENNING JAFNHRAÐAHREYFLA Í III. ÁFANGA A (HREYFILFLOKKAR: H, I, J og K)

3c.GERÐARVIÐURKENNING HREYFLA Í III. ÁFANGA B (HREYFILFLOKKAR: L, M, N og P)

3d.GERÐARVIÐURKENNING HREYFLA Í IV. ÁFANGA (HREYFILFLOKKAR:

3e.GERÐARVIÐURKENNING KNÚNINGSVÉLA Í III. ÁFANGA A, SEM NOTAÐAR ERU Í SKIP Í SIGLINGUM Á SKIPGENGUM VATNALEIÐUM (HREYFILFLOKKUR V)

3f.GERÐARVIÐURKENNING KNÚNINGSVÉLA Í III. ÁFANGA A, SEM NOTAÐAR ERU Í SPORREIÐAR

3g.GERÐARVIÐURKENNING KNÚNINGSVÉLA Í III. ÁFANGA B, SEM NOTAÐAR ERU Í SPORREIÐAR

3h.GERÐARVIÐURKENNING KNÚNINGSVÉLA Í III. ÁFANGA A, SEM NOTAÐAR ERU Í EIMREIÐAR

3i.GERÐARVIÐURKENNING KNÚNINGSVÉLA Í III. ÁFANGA B, SEM NOTAÐAR ERU Í EIMREIÐAR

II.KAFLI

10.Ákvæði 1. -- 9. mgr. gilda með fyrirvara um ákvæði sem bandalagið samþykkir um vörumerkingu.



In [44]:
from concurrent.futures import ProcessPoolExecutor
from functools import partial
from tqdm import tqdm
from frontend import core as c

regexps = [{
    'pattern': re.compile(r'([\w\(\)\[\]\.]{2,})\.([A-ZÁÐÉÍÓÚÝÞÆÖ])'),
    'repl': r'\1. \2'
}]

def apply_regexp_parallel(regexps, corpus, threads):
    with ProcessPoolExecutor(max_workers=threads) as executor:
        results = tqdm(executor.map(
            partial(c.regexp, regexps=regexps),
            corpus,
            chunksize=2000),
            total=len(corpus))
        return [result for result in results]

In [45]:
train_sent_fix_is = apply_regexp_parallel(regexps, train_is, 6)
train_sent_fix_en = apply_regexp_parallel(regexps, train_en, 6)

100%|██████████| 3256945/3256945 [00:26<00:00, 124481.75it/s]


In [46]:
print_indices(train_sent_fix_is, sent_break_indices)

3b. GERÐARVIÐURKENNING JAFNHRAÐAHREYFLA Í III. ÁFANGA A (HREYFILFLOKKAR: H, I, J og K)

3c. GERÐARVIÐURKENNING HREYFLA Í III. ÁFANGA B (HREYFILFLOKKAR: L, M, N og P)

3d. GERÐARVIÐURKENNING HREYFLA Í IV. ÁFANGA (HREYFILFLOKKAR:

3e. GERÐARVIÐURKENNING KNÚNINGSVÉLA Í III. ÁFANGA A, SEM NOTAÐAR ERU Í SKIP Í SIGLINGUM Á SKIPGENGUM VATNALEIÐUM (HREYFILFLOKKUR V)

3f. GERÐARVIÐURKENNING KNÚNINGSVÉLA Í III. ÁFANGA A, SEM NOTAÐAR ERU Í SPORREIÐAR

3g. GERÐARVIÐURKENNING KNÚNINGSVÉLA Í III. ÁFANGA B, SEM NOTAÐAR ERU Í SPORREIÐAR

3h. GERÐARVIÐURKENNING KNÚNINGSVÉLA Í III. ÁFANGA A, SEM NOTAÐAR ERU Í EIMREIÐAR

3i. GERÐARVIÐURKENNING KNÚNINGSVÉLA Í III. ÁFANGA B, SEM NOTAÐAR ERU Í EIMREIÐAR

II. KAFLI

10. Ákvæði 1. -- 9. mgr. gilda með fyrirvara um ákvæði sem bandalagið samþykkir um vörumerkingu.



## Laga greina vísanir og styttingar
Á þó nokkrum stöðunum í textanum vantar bil og punkt eftir númerum sem og styttingar á orðum klikka. Skoðum nokkur slík dæmi og reynum að laga þau. Sama vandamál er ekki í enska textanum.

In [47]:
ref_indices = find_indices_match(train_sent_fix_is, re.compile(r"(\d+(.\d+)?)(mgr|gr|skv|og|eða|til|með|janúar|febrúar|mars|apríl|maí|júní|júlí|ágúst|september|október|nóvember|desember)"), max_count=10)
print_indices(train_sent_fix_is, ref_indices)
print_indices(train_sent_fix_en, ref_indices)

1mgr56grkomi tilvísun í „9., 10og 10gra“ í stað tilvísunar í „9og 10gr.“

2gr.,

2gr.

1grer breytt sem hér segir:

3mgr2grfalli niður;

6mgr6grfalli niður að því er varðar árið 1999.

1til 3mgr16gr.

4og 5grfalli brott3gr.

in Article 56(1) the reference to ‘Articles 9, 10 shall be replaced by a reference to ‘Articles 9, 10, 10a;

Article 2;

Article 2:

Article 1 shall be amended as follows:

Article 2(3) shall be deleted;

for 1999, paragraph 6 of Article 6 shall be deleted.

(1) third subparagraph

Articles 4 and 5 shall be deleted.



In [48]:
regexps = [
    {
        'pattern': re.compile(r"(\d+(.\d+)?)(mgr|gr|skv|og|eða|til|með|janúar|febrúar|mars|apríl|maí|júní|júlí|ágúst|september|október|nóvember|desember)"),
        'repl': r"\1. \3"
    },
    {
        'pattern': re.compile(r"(mgr|gr|skv)(?=[^\.])"),
        'repl': r"\1. "
    }
]
train_ref_fix_is = apply_regexp_parallel(regexps, train_sent_fix_is, 6)
train_ref_fix_en = train_sent_fix_en

100%|██████████| 3256945/3256945 [00:28<00:00, 115881.53it/s]


In [49]:
print_indices(train_ref_fix_is, ref_indices)

1. mgr. 56. gr. komi tilvísun í „9., 10. og 10. gr. a“ í stað tilvísunar í „9. og 10. gr.“

2. gr.,

2. gr.

1. gr. er breytt sem hér segir:

3. mgr. 2. gr. falli niður;

6. mgr. 6. gr. falli niður að því er varðar árið 1999.

1. til 3. mgr. 16. gr.

4. og 5. gr. falli brott3. gr.



## Lesa síuð prófunargögn
Til þess að geta borið saman niðurstöður beint við Miðeind þá notum við síað prófunarsett frá þeim. Þá er búið að fjarlægja tvítekningar og setningar sem innihalda stafi úr "óþekktum" stafrófum.

In [50]:
!ls {source_test_dir}/

filtered  raw


In [51]:
test_filtered_ees_is = read_file(source_test_dir.joinpath('filtered').joinpath('is').joinpath('ees.is'))
test_filtered_ees_en = read_file(source_test_dir.joinpath('filtered').joinpath('en').joinpath('ees.en'))
test_filtered_ema_is = read_file(source_test_dir.joinpath('filtered').joinpath('is').joinpath('ema.is'))
test_filtered_ema_en = read_file(source_test_dir.joinpath('filtered').joinpath('en').joinpath('ema.en'))
test_filtered_opensubtitles_is = read_file(source_test_dir.joinpath('filtered').joinpath('is').joinpath('opensubtitles.is'))
test_filtered_opensubtitles_en = read_file(source_test_dir.joinpath('filtered').joinpath('en').joinpath('opensubtitles.en'))
print(len(test_filtered_ees_is), 
      len(test_filtered_ees_en), 
      len(test_filtered_ema_is),
      len(test_filtered_ema_en),
      len(test_filtered_opensubtitles_is),
      len(test_filtered_opensubtitles_en))

1930 1930 1963 1963 2059 2059


Skoðum aðeins hver munurinn er.

In [52]:
filtered_set = frozenset(test_filtered_ees_is)
original_set = frozenset(test_ees_is)

In [35]:
removed_sentences = original_set.difference(filtered_set)
removed_sentences

frozenset({'"1 NOEC á fisk, daphnia eða þörunga"1 (NOEC ef viðkomandi tegund\n',
           '"2 NOEC á fisk, daphnia eða þörunga5\n',
           '"3 L50 á fisk, daphnia eða þörunga"20 (lægsti L50)\n',
           '"Minnst 1 L50 á fisk, daphnia eða þörunga"50 (lægsti eða 20 í sérstökum tilvikum (sbr. hér að aftan)"\n',
           '"Minnst 2 NOEC á fisk, daphnia eða þörunga"1 (lægsti NOEC)\n',
           '"Nota ber lægsta staðfesta NOEC"\n',
           '"a"og"b"eru stuðlar sem eru fastsettir fyrir hvern nákvæmnisflokk samkvæmt eftirfarandi töflu:m]\n',
           '"frá tilteknum þriðju löndum sem ekki eru skráð í viðauka A við ákvörðun framkvæmdastjórnarinnar 97/198/EB.";\n',
           '"Það sparar orku að þvo við lágan hita og með því að nota lítinn skammt af þvottaefni má draga úr hugsanlegum umhverfisspjöllum."\n',
           '(*) Þessi undanþága gildir að því tilskildu að yfirborðsvirk efni séu í samræmi við a-lið 3. viðmiðunar og að þau séu niðurbrjótanleg við loftfirrðar aðstæður.\

Það virðist vera sem svo að setningarnar s sem innihalda "-/^", óþekkt orð (?), óþekkta stafi, mikið af merkjum og stuttar setningar.

## Skrifa niðurstöður
Núna skrifum við lokaniðurstöðurnar í skrá.

In [53]:
def write_to_file(corpus, path):
    with path.open('w+') as f_in:
        for line in corpus:
            f_in.write(line)
    !wc -l {path}

In [54]:
write_to_file(train_ref_fix_en, target_dir.joinpath('train.en'))
write_to_file(train_ref_fix_is, target_dir.joinpath('train.is'))
write_to_file(dev_en, target_dir.joinpath('dev.en'))
write_to_file(dev_is, target_dir.joinpath('dev.is'))
write_to_file(test_filtered_ees_en, target_dir.joinpath('test-ees.en'))
write_to_file(test_filtered_ees_is, target_dir.joinpath('test-ees.is'))
write_to_file(test_filtered_ema_en, target_dir.joinpath('test-ema.en'))
write_to_file(test_filtered_ema_is, target_dir.joinpath('test-ema.is'))
write_to_file(test_filtered_opensubtitles_en, target_dir.joinpath('test-opensubtitles.en'))
write_to_file(test_filtered_opensubtitles_is, target_dir.joinpath('test-opensubtitles.is'))

3256945 /work/haukurpj/data/filtered/Parice1.0/train.en
3256945 /work/haukurpj/data/filtered/Parice1.0/train.is
2000 /work/haukurpj/data/filtered/Parice1.0/dev.en
2000 /work/haukurpj/data/filtered/Parice1.0/dev.is
1930 /work/haukurpj/data/filtered/Parice1.0/test-ees.en
1930 /work/haukurpj/data/filtered/Parice1.0/test-ees.is
1963 /work/haukurpj/data/filtered/Parice1.0/test-ema.en
1963 /work/haukurpj/data/filtered/Parice1.0/test-ema.is
2059 /work/haukurpj/data/filtered/Parice1.0/test-opensubtitles.en
2059 /work/haukurpj/data/filtered/Parice1.0/test-opensubtitles.is
